In [1]:
import numpy as np

In [2]:
def tucker_to_tensor(core, factors):
    fold_part = ','.join([chr(ord('a') + i) + chr(ord('A') + i) for i in range(len(core.shape))])
    common_part = ''.join([chr(ord('A') + i) for i in range(len(core.shape))])
    T = np.einsum(common_part + ',' + fold_part, core, *factors)
    return T

In [3]:
def truncated_SVD(A, delta):
    u, s, v = np.linalg.svd(A, full_matrices=False)
    rank = (np.abs(s) >= delta * s[0]).sum()
    return u[:, :rank], s[:rank], v[:rank, :], rank

In [4]:
def st_HOSVD(tensor, eps=1e-15):
    G = tensor.copy()
    dims = list(G.shape)
    d = len(dims)
    S = [None for _ in range(d)]
    singulars = [None for _ in range(d)]

    for k in range(d):
        G = G.reshape(G.shape[0], -1)

        S[k], singulars[k], G, dims[k] = truncated_SVD(G, eps)
        G = G[: dims[k], :]
        G = G.reshape(dims[k:] + dims[:k])
        G = np.moveaxis(G, 0, -1)

    for j in range(d):
        G = np.moveaxis(np.moveaxis(G, j, -1) * singulars[j], -1, j)
    return G, S

In [5]:
def sum_tuckers(core1, factors1, core2, factors2):
    shape1 = core1.shape
    shape2 = core2.shape

    factors3 = [np.hstack((factors1[i], factors2[i])) for i in range(len(shape1))]

    core3 = np.zeros(np.array(shape1) + np.array(shape2))

    inds1 = (slice(0, shape1[0]),)
    for i in range(1, len(shape1)):
        inds1 += (slice(0, shape1[i]),)
    inds2 = (slice(shape1[0], shape1[0] + shape2[0]),)
    for i in range(1, len(core2.shape)):
        inds2 += (slice(shape1[i], shape1[i] + shape2[i]),)

    core3[inds1] = core1
    core3[inds2] = core2

    return core3, factors3

In [6]:
def squeeze(core, factors, eps=1e-15):
    shape = core.shape
    d = len(shape)
    R = [None for _ in range(d)]
    for i in range(d):
        factors[i], R[i] = np.linalg.qr(factors[i], mode='reduced')
    T = tucker_to_tensor(core, R)
    core, new_factors = st_HOSVD(T, eps)
    R = [factors[i] @ new_factors[i] for i in range(len(core.shape))]

    return core, R

# Tests

In [7]:
def f1(i, j, k):
    return np.sin(i + j + k)

In [8]:
T = np.fromfunction(f1, (32, 64, 128))

In [9]:
G, S = st_HOSVD(T, eps=1e-13)
print(f'Kernel shape: {G.shape}')

Kernel shape: (2, 2, 2)


In [10]:
T_new = tucker_to_tensor(G, S)
err = np.linalg.norm(T_new - T) / np.linalg.norm(T)
print(f'Relative error: {err}')

Relative error: 3.956674474255315e-15


In [11]:
def f2(i, j, k):
    return np.cos(i + j + k)

In [12]:
def f3(i, j, k):
    return f1(i, j, k) + f2(i, j, k)

In [13]:
shape = (100, 100, 100)
T1 = np.fromfunction(f1, shape)
T2 = np.fromfunction(f2, shape)
T3 = np.fromfunction(f3, shape)

In [14]:
eps = 1e-13

In [15]:
G1, S1 = st_HOSVD(T1, eps)
G2, S2 = st_HOSVD(T2, eps)
print(f'First tensor kernel shape:  {G1.shape}')
print(f'Second tensor kernel shape: {G2.shape}')

First tensor kernel shape:  (2, 2, 2)
Second tensor kernel shape: (2, 2, 2)


In [16]:
G3, S3 = sum_tuckers(G1, S1, G2, S2)
print(f'Sum of tensors kernel shape: {G3.shape}')

Sum of tensors kernel shape: (4, 4, 4)


In [17]:
G4, S4 = squeeze(G3, S3, eps)
print(f'Kernel shape after squeeze: {G4.shape}')

Kernel shape after squeeze: (2, 2, 2)


In [18]:
T_res = tucker_to_tensor(G4, S4)
err = np.linalg.norm(T3 - T_res) / np.linalg.norm(T3)
print(f'Relative error: {err}')

Relative error: 3.3820190412907313e-15
